In [6]:
import tensorflow as tf
import keras
from keras.applications.inception_resnet_v2 import InceptionResNetV2
import keras.utils as image
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input, decode_predictions
import numpy as np
from keras.models import Model,Sequential
from tensorflow.keras.layers import *
from tensorflow.keras import optimizers
import pandas as pd 
import cv2
import os 
import matplotlib.pyplot as plt
from tensorflow.keras.metrics import Precision,Recall
from skimage.metrics import structural_similarity
import csv
from ultralytics import YOLO
import datetime
import ffmpeg

In [7]:
file_dir = './video_files/S2_T5_Shivam/'
video_file = '../Videos/S2/T5/S2_T5_Shivam.mp4'
model_path = './manoj_code/yolov8m_custom.pt'
w_min_ratio = 0
w_max_ratio = 1
h_min_ratio =0.11
h_max_ratio =0.79

In [8]:
def format_timestamp(timestamp):
  """Converts a timestamp in seconds to the format hh:mm:ss."""
  hours = int(timestamp // 3600)
  minutes = int((timestamp % 3600) // 60)
  seconds = int(timestamp % 60)

  return f'{hours:02d}:{minutes:02d}:{seconds:02d}'

# Load the YOLOv8 model
model = YOLO(model_path)

# Open the video file
cap = cv2.VideoCapture(video_file)

f = cap.get(cv2.CAP_PROP_FPS)
fps = int(round(f))

timestamps = []
frame_number=0

In [9]:
# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if not success:
       cap.release()
       break
       
    if(frame_number%fps == 0):
        # Run YOLOv8 inference on the frame
       results = model(frame)

       if len(results[0]) != 0:
        # Calculate timestamp based on frame number and fps
        timestamp = frame_number / fps

        # Format the timestamp
        formatted_timestamp = format_timestamp(timestamp)

        # Append the timestamp to the list
        timestamps.append(formatted_timestamp)
    
    # Increment the frame number
    frame_number += 1

# Release the video capture object and close the display window
cap.release()
# Open the output file for writing
txt_file = open(file_dir+'/timestamps.txt', 'w')

# Write the timestamps to the file
for timestamp in timestamps:
  txt_file.write(timestamp + '\n')

# Close the output file
txt_file.close()
cv2.destroyAllWindows()


0: 384x640 (no detections), 209.3ms
Speed: 0.0ms preprocess, 209.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Whiteboard, 198.8ms
Speed: 0.0ms preprocess, 198.8ms inference, 15.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Whiteboard, 234.4ms
Speed: 0.0ms preprocess, 234.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Whiteboard, 206.8ms
Speed: 0.0ms preprocess, 206.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Whiteboard, 237.6ms
Speed: 0.0ms preprocess, 237.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Whiteboard, 218.8ms
Speed: 0.0ms preprocess, 218.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Whiteboard, 236.7ms
Speed: 0.0ms preprocess, 236.7ms inference, 15.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Whiteboard, 156.3ms
Speed: 0.0ms preprocess, 156.3ms infe

In [10]:
def open_file_and_extract_contents_in_list(filename):
    """Opens a file and extracts its contents in a list.

    Args:
        filename: The path to the file to open.

    Returns:
        A list containing the contents of the file.
    """

    with open(filename, "r") as f:
        contents = f.read()

    list_of_contents = contents.split("\n")

    return list_of_contents

In [11]:
def convert_hhmmss_to_seconds(time_string):
  """Converts a string of hh:mm:ss to seconds.

  Args:
    time_string: A string of hh:mm:ss.

  Returns:
    The number of seconds in the time string.
  """
  
  
  hours, minutes, seconds = time_string.split(":")
  hours = int(hours)
  minutes = int(minutes)
  seconds = int(seconds)

  total_seconds = hours * 3600 + minutes * 60 + seconds

  return total_seconds

# Example usage:

list_of_contents = open_file_and_extract_contents_in_list(file_dir+"/timestamps.txt")

list_of_contents.pop()

size=len(list_of_contents)

new_list = []

if size==1:
   new_list.append((list_of_contents[0],list_of_contents[0]))

else:
   k=0
   while True:
        t1=list_of_contents[k]
        t2=t1
        flag = True
        for x in range(k+1,size):
            t3=convert_hhmmss_to_seconds(list_of_contents[x])
            if (t3-convert_hhmmss_to_seconds(t2))<=1:
                t2=list_of_contents[x]
            else:
                new_list.append((list_of_contents[k],t2))
                k=x
                flag=False
                break
        
        if flag:
            new_list.append((t1,t2))
            break

In [12]:
def write_list_of_pairs_to_csv_file(filename, list_of_pairs):
  """Writes a list of pairs to a CSV file.

  Args:
    filename: The path to the CSV file to write to.
    list_of_pairs: The list of pairs to write to the CSV file.
  """
  with open(filename, "w", newline="") as f:
    writer = csv.writer(f)
    for pair in list_of_pairs:
      writer.writerow(pair)



timestamps_csv = file_dir + "/timestamps.csv"
write_list_of_pairs_to_csv_file(timestamps_csv,new_list)

In [13]:
df = pd.read_csv(timestamps_csv)
arr = df.to_numpy()

In [14]:
def string_to_sec(start_time):
   hours = start_time[0:2]
   minutes = start_time[3:5]
   seconds = start_time[6:8]
   
   if(hours[0]=='0'):
      hours = hours[1]

   if(minutes[0]=='0'):
      minutes = minutes[1]

   if(seconds[0]=='0'):
      seconds =seconds[1]
 
   hours = int(hours)
   minutes =int(minutes)
   seconds =int(seconds)

   sec = (hours*3600 + minutes*60 + seconds)

   return sec

In [15]:
filtered_start_time = []
filtered_end_time = []

i= 0 
while i in range(0,df.shape[0]):
    
    if(string_to_sec(arr[i][1]) - string_to_sec(arr[i][0]) > 20):

      filtered_start_time.append(arr[i][0])
      filtered_end_time.append(arr[i][1])
    i+=1

filtered_start_time
filtered_end_time

fields = ['start_time','end_time'] 
timings_file = file_dir+'/clip_timings.csv'

with open(timings_file, 'w') as csvfile: 
    
    csvwriter = csv.writer(csvfile)# creating a csv writer object 
    
    csvwriter.writerow(fields)  

df= pd.read_csv(timings_file)

df['start_time'] = filtered_start_time
df['end_time'] = filtered_end_time
df.to_csv(timings_file ,index=False)

In [16]:
source_path = video_file
destination_path = file_dir + "/wb_clip"
df = pd.read_csv(timings_file)
arr = df.to_numpy()
cap = cv2.VideoCapture(source_path) 
frame_per_sec = int(round(cap.get(cv2.CAP_PROP_FPS)))

In [17]:
def video_trim(source_path,destination_path,start_time,end_time,clip_count):
    (
	ffmpeg.input(source_path, ss=start_time, to=end_time)
	.filter('fps', fps=frame_per_sec, round='up')
	.output(destination_path+"_{}.mp4".format(str(clip_count)))
	.run()
    ) 

In [18]:
for i in range(0,df.shape[0]):
    start_time = str(arr[i][0])
    end_time = str(arr[i][1])
    video_trim(source_path,destination_path,start_time,end_time,i+1)

In [19]:
physical_devices =tf.config.list_physical_devices ('GPU') # setting computng device to GPU
tf.config.experimental.set_memory_growth(physical_devices[0],True) # setting the memory growth of GPU consumption  so that all memory is not allotted and is finished

In [20]:
# making the concat layer which is build on top of the inception model 
class L2Concat(Layer):
    def __init__(self,**kwargs):
        super().__init__()

    def call(self,image_1_embedding,image_2_embedding):
        return  tf.concat([image_1_embedding,image_2_embedding],-1) # concatenates the features of the image pairs as the last layer 

In [21]:
model =tf.keras.models.load_model('siamesemodel_multi.h5',compile=False,custom_objects={'L2Concat':L2Concat,'SparseCategoricalCrossentropy':tf.losses.SparseCategoricalCrossentropy}) # loading the model 

In [22]:
def compare_frame(frameA, frameB):
  grayA = cv2.cvtColor(frameA, cv2.COLOR_BGR2GRAY) # convert to grayscale
  grayB = cv2.cvtColor(frameB, cv2.COLOR_BGR2GRAY) # convert to grayscale
   
  score, diff = structural_similarity(grayA, grayB, full=True) # find SSIM of the image pairs 
  diff = (diff * 255).astype("uint8")
  
  print("SSIM: {}".format(score)) # prints similarity score 

  thresh = cv2.threshold(diff, 180, 255, cv2.THRESH_BINARY_INV)[1]    # if pixel values is greater than threshold (180) mapped to 0, else to 255 (THRESH_BINARY_INV)
  cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # find the binary contours
  cnts=cnts[0] # returns the list of countour boundaries 

  return diff, thresh, cnts, score

def convert_box(cnts):
  box = []
  for c in cnts:
    x, y, w, h = cv2.boundingRect(c) # put a bounding box in contour areas
    if w > 5 and h > 16: 
      box.append([x, y, x+w, y+h]) # checks if change has happended in area greater than 16px * 5px size  ,and appends them to the array of box coordinates 
  box = np.array(box) #  making the array of boxes to a numpy array 
  # print(box)
  return box

def non_max_suppression(boxes, overlapThresh):
  if len(boxes) == 0:
    return []
  if boxes.dtype.kind == "i":  
    boxes = boxes.astype("float")  # checks the data type of the boxed array that is passed and if an integer makes it to float

  pick = []
  x1 = boxes[:,0]
  y1 = boxes[:,1]
  x2 = boxes[:,2]
  y2 = boxes[:,3]
  area = (x2 - x1 + 1) * (y2 - y1 + 1) # computes area of all boxes  using the  coordinates available 
  idxs = np.argsort(y2) # returns the indexes in ascending sorted order (it according to top to bottom depending on y axis) 

  while len(idxs) > 0:
   last = len(idxs) - 1
   i = idxs[last]
   pick.append(i)
   xx1 = np.maximum(x1[i], x1[idxs[:last]])
   yy1 = np.maximum(y1[i], y1[idxs[:last]])
   xx2 = np.minimum(x2[i], x2[idxs[:last]])
   yy2 = np.minimum(y2[i], y2[idxs[:last]])
   w = np.maximum(0, xx2 - xx1 + 1)
   h = np.maximum(0, yy2 - yy1 + 1)
   overlap = (w * h) / area[idxs[:last]] 
   idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlapThresh)[0]))) # removes the overlapping boxes  and has boxes with less than 0.3 threshold factor intersection

  return boxes[pick].astype("int")
  


def find_max(boxes_nms):
  if len(boxes_nms) == 0:
    return []
  boxes = []
  for box_nms in boxes_nms:
    box_nms = np.append(box_nms, (box_nms[2]-box_nms[0])*(box_nms[3]-box_nms[1]))
    boxes.append(box_nms) 
  boxes = np.array(boxes) # gets the area of the boxes and is appended as andditional column
  idx = np.argsort(boxes[:,4]) # sorts based on area 
  x_center = boxes[idx[-1]][0] + (boxes[idx[-1]][2] - boxes[idx[-1]][0]) / 2  
  y_center = boxes[idx[-1]][1] + (boxes[idx[-1]][3] - boxes[idx[-1]][1]) / 2
  box_max = np.append(boxes[idx[-1]], [x_center, y_center]) # stores coordinates, cneter points of x,y and area of the largest
  box_max = np.array(box_max, dtype = np.int32)
  return box_max

def find_max_region(boxes_nms):
  if len(boxes_nms) == 0:
    return []
  x1 = boxes_nms[:,0]
  y1 = boxes_nms[:,1]
  x2 = boxes_nms[:,2]
  y2 = boxes_nms[:,3]  
  xx1 = min(x1)
  yy1 = min(y1)
  xx2 = max(x2)
  yy2 = max(y2)
  max_region = np.array([xx1, yy1, xx2, yy2]) # computes the coordinates of  maximum sized box that can include the largest share of changes
  return max_region

def to_canvas(region):
  canvas = np.zeros((input_border_length, input_border_length, 3), np.uint8) # defines a canvas of 299x 299 
  print(region.shape)
  if region.shape[0] > region.shape[1]:
    if region.shape[1] % 2:
      canvas[:, int(input_border_length / 2 - (region.shape[1] / 2)):int(input_border_length / 2 + (region.shape[1] / 2) + 1)-1] = region
    else:
      canvas[:, int(input_border_length / 2 - (region.shape[1] / 2)):int(input_border_length / 2 + (region.shape[1] / 2))] = region
  else:
    if region.shape[0] % 2:
      canvas[int(input_border_length/ 2 - (region.shape[0] / 2)):int(input_border_length / 2 + (region.shape[0] / 2) + 1)-1, :] = region
    else:
      canvas[int(input_border_length / 2 - (region.shape[0] / 2)):int(input_border_length / 2 + (region.shape[0] / 2)), :] = region
  return canvas

In [23]:
# function to the predict the label of image pairs 
def tag(img_A,img_B):
 probs = model.predict(list(np.expand_dims([img_A,img_B],axis=1)))
 return np.argmax(probs) # gets the label (index is used) 

In [24]:
def change_region(frame_no_A,frame_no_B):
 
 cap = cv2.VideoCapture(video_path) #reads video from the path 
 w_frame, h_frame = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # computes the dimension of the video frame
 h_min=round(h_min_ratio*h_frame) 
 h_max=round(h_max_ratio*h_frame) # cropping the area of interest
 w_min=round(w_min_ratio*w_frame)
 w_max=round(w_max_ratio*w_frame)
 w=w_max-w_min
 h=h_max-h_min

 cap.set(1,frame_no_A) # sets the video in the required frame number 
 ret,frame1 = cap.read() 
 frameA = frame1[h_min:h_max, w_min:w_max] # reads  the area of interest 

 cap.set(1,frame_no_B) # sets the video in the required frame number 
 ret,frame2 = cap.read() 
 frameB = frame2[h_min:h_max, w_min:w_max] # reads the area of interest 
 
 diff, thresh, cnts, score= compare_frame(frameA, frameB) # gets the image differences,threshold value
 mask_region = np.zeros((frameB.shape[0], frameB.shape[1]), np.uint8) #  produces a black image of the dimensions of the frame
 boxes = convert_box(cnts) 
 boxes_nms = non_max_suppression(boxes, 0.3) # removes overlaps 
 max_box = find_max(boxes_nms) # optional code
 max_region_box = find_max_region(boxes_nms) 
 for box in boxes_nms:
    cv2.rectangle(mask_region, (box[0], box[1]), (box[2], box[3]), (255, 255, 255), -1) # draws rectangles on the non intersecting boxes

 frameC = frameB.copy()
 frameD = frameB.copy()
 label =-1

 if len(boxes_nms):
    cv2.rectangle(frameC, (max_region_box[0], max_region_box[1]), (max_region_box[2], max_region_box[3]), (0, 255, 0), 2) # draws a rectangle
    max_region_A = frameA[max_region_box[1]:max_region_box[3], max_region_box[0]:max_region_box[2]].copy() # applies the rectangle box mask on frameA
    mask_region = mask_region[max_region_box[1]:max_region_box[3], max_region_box[0]:max_region_box[2]].copy()
    max_region_D = frameD[max_region_box[1]:max_region_box[3], max_region_box[0]:max_region_box[2]].copy() # applies rectangle box mask on frameB
    region_A = cv2.bitwise_and(max_region_A, max_region_A, mask = mask_region)
    region_D = cv2.bitwise_and(max_region_D, max_region_D, mask = mask_region) # compute the bitwise_and to get the image cutout
    f_rate =  input_border_length* 1.0 / max(region_A.shape[0], region_A.shape[1]) # scaling the image propotionally 
    region_A = cv2.resize(region_A, (0,0), fx = f_rate, fy = f_rate) 
    region_D = cv2.resize(region_D, (0,0), fx = f_rate, fy = f_rate) 
    img_A = to_canvas(region_A)
    img_D = to_canvas(region_D)

    ## Model predictions 
    label = tag(img_A,img_D)
    ## figuring out the index label 
    ## classification based on label 
    
    ## to csv 
 else:
    
    label =2
 return label


In [25]:
# code to convert milliseconds to seconds
def mil_convert(milliseconds):
  seconds, milliseconds = divmod(milliseconds, 1000)
  minutes, seconds = divmod(seconds, 60)
  hours ,minutes =divmod(minutes,60)
  
  hours_str   =  str(round(hours))
  minutes_str = str(round(minutes))
  seconds_str = str(round(seconds))

  if(round(hours)<10):
    hours_str = "0" + hours_str
  
  if(round(minutes)<10):
    minutes_str = "0" + minutes_str

  if(round(seconds)<10):
    seconds_str = "0" + seconds_str  

  time = hours_str +":"+ minutes_str +":"+seconds_str
  return time

In [26]:
def string_to_millis(start_time):
   hours = start_time[0:2]
   minutes = start_time[3:5]
   seconds = start_time[6:8]
   
   if(hours[0]=='0'):
      hours = hours[1]

   if(minutes[0]=='0'):
      minutes = minutes[1]

   if(seconds[0]=='0'):
      seconds =seconds[1]
 
   hours = int(hours)
   minutes =int(minutes)
   seconds =int(seconds)

   millis = (hours*3600 + minutes*60 + seconds)*1000

   return millis 

In [27]:
res = []
input_border_length =299 
vosaic_buffer = "00:00:00"

# Iterate directory
for file in os.listdir(file_dir):
    # check only text files
    if file.endswith('.mp4'):
        res.append(file)

res = sorted(res, key=lambda x: int(x[8:-4]))        

df_1 = pd.read_csv(file_dir+'/clip_timings.csv')
arr_1 = df_1.to_numpy()

for i in range(len(res)):

   clip_id = i+1
   fields = ['timestamp_A','timestamp_B','action_code'] # make required columns in the csv file
   report_file = file_dir+"/{}.csv".format(str(res[clip_id-1][0:-4]))   # file name of the csv
   video_start =string_to_millis(arr[clip_id-1][0]) + string_to_millis(vosaic_buffer)
   with open(report_file, 'w') as csvfile: 
    
      csvwriter = csv.writer(csvfile)# creating a csv writer object 
    
      csvwriter.writerow(fields)  

   video_path = file_dir + res[clip_id-1]
   cap = cv2.VideoCapture(video_path) # reading testing video
   frame_count= int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) # frame counts
   fps    = cap.get(cv2.CAP_PROP_FPS)
   # loop to analyse frame data and write actions to 
   j=0
   while(int(fps)*(j+1)<int(frame_count)):
 
     action_code = change_region(int(fps)*(j),int(fps)*(j+1)) # passes a frame pair ,one from the beginning of a 1 second interval and other from the end of a the interval
     timestamp_A =  mil_convert((j)*1000+video_start)
     timestamp_B =  mil_convert((j+1)*1000+video_start)
     data = { 
             'timestamp_A' : [timestamp_A],  
             'timestamp_B' : [timestamp_B],
             'action_code': [action_code],
            }    
     df = pd.DataFrame(data)
     df.to_csv(report_file,mode ='a' , index=False, header=False)  # write the data into the csv
 
     j+=1  


SSIM: 0.9992037123971501
(299, 245, 3)
(299, 245, 3)
1/1 [==============================] - 19s 19s/step
SSIM: 0.9989595820379881
(299, 269, 3)
(299, 269, 3)
1/1 [==============================] - 0s 78ms/step
SSIM: 0.9997973010854571
SSIM: 0.9999156022655352
SSIM: 0.999968887619569
SSIM: 0.9998793053998436
SSIM: 0.999876964129103
SSIM: 0.999404915622909
(299, 299, 3)
(299, 299, 3)
1/1 [==============================] - 0s 78ms/step
SSIM: 0.9947478210547192
SSIM: 0.9999039645458891
SSIM: 0.9999501203999807
SSIM: 0.9999630367268787
SSIM: 0.9999412869671426
SSIM: 0.9999345142344085
SSIM: 0.9999779782733637
SSIM: 0.9999060868630508
SSIM: 0.9974527014080113
SSIM: 0.994329036014066
(72, 299, 3)
(72, 299, 3)
1/1 [==============================] - 0s 94ms/step
SSIM: 0.9896168639287305
(21, 299, 3)
(21, 299, 3)
1/1 [==============================] - 0s 62ms/step
SSIM: 0.9931729899118589
(67, 299, 3)
(67, 299, 3)
1/1 [==============================] - 0s 62ms/step
SSIM: 0.9900165411228142
(147,

In [28]:
csv_list =[]
    
for file in os.listdir(file_dir):
    # check only text files
    if file.startswith('wb_clip') and file.endswith('.csv') :
      csv_list.append(file)

csv_list = sorted(csv_list, key=lambda x: int(x[8:-4]))  

other_activity_time =0
sketch_time = 0
no_activity_time = 0


for i in range (0,len(csv_list)):
 location = file_dir+"/{}.csv".format(str(csv_list[i][0:-4])) 
 df = pd.read_csv(location)
 arr =df.to_numpy()
 sketch =0
 other = 0
 no_activity = 0

 for j in range (0,df.shape[0]):
   if((arr[j][2]==0)):
    other+=1 
   elif ((arr[j][2]==1)):
    sketch +=1
   elif ((arr[j][2]==2)):
    no_activity+=1

 sketch_time+=sketch
 other_activity_time+=other
 no_activity_time+=no_activity

In [29]:
csv_list

['wb_clip_1.csv',
 'wb_clip_2.csv',
 'wb_clip_3.csv',
 'wb_clip_4.csv',
 'wb_clip_5.csv',
 'wb_clip_6.csv',
 'wb_clip_7.csv',
 'wb_clip_8.csv',
 'wb_clip_9.csv',
 'wb_clip_10.csv',
 'wb_clip_11.csv',
 'wb_clip_12.csv',
 'wb_clip_13.csv',
 'wb_clip_14.csv',
 'wb_clip_15.csv',
 'wb_clip_16.csv',
 'wb_clip_17.csv',
 'wb_clip_18.csv',
 'wb_clip_19.csv',
 'wb_clip_20.csv',
 'wb_clip_21.csv',
 'wb_clip_22.csv',
 'wb_clip_23.csv',
 'wb_clip_24.csv',
 'wb_clip_25.csv',
 'wb_clip_26.csv',
 'wb_clip_27.csv',
 'wb_clip_28.csv',
 'wb_clip_29.csv',
 'wb_clip_30.csv',
 'wb_clip_31.csv',
 'wb_clip_32.csv',
 'wb_clip_33.csv',
 'wb_clip_34.csv',
 'wb_clip_35.csv',
 'wb_clip_36.csv']

In [30]:
fields = ['sketch_duration','other_activity_duration' ,'no_activity_duration']
analysis_file = file_dir+"/{}.csv".format('analysis_file') 
with open(analysis_file, 'w') as csvfile: 
    
      csvwriter = csv.writer(csvfile)# creating a csv writer object 
    
      csvwriter.writerow(fields) 

df =pd.read_csv(analysis_file)
data = { 
        'sketch_duration' : [mil_convert(sketch_time*1000)],  
        'other_activity_duration' : [mil_convert(other_activity_time*1000)],
        'no_activity_duration': [mil_convert(no_activity_time*1000)],
        }    
df = pd.DataFrame(data)
df.to_csv(analysis_file,mode ='a' , index=False, header=False) 
